In [1]:
import numpy as np
import pandas as pd
import data_pipeline2 as dp
import seaborn as sns
from test_fncs import recursive_pred
import torch.nn as nn
import torch

import training_models as tm
import ml_models


from test_fncs import recursive_pred
from sklearn.metrics import r2_score

In [2]:
df_train, df_test = dp.get_cmapss_data(0, 80, data_dir ='data_synthetic', chosen_sensor = 11)
output_sizes = [5,10,20,40]
input_sizes = [5,10,20,40]

config = {
    'input_size' : 30,
    'lr' : 1e-4,
    'batch_size' : 32,
    'hidden_size' : 8,
    'output_size' : 3,
    'layer_amt' : 3
    }

loss = nn.MSELoss()

In [3]:


results = []

for input_size in input_sizes:
    for output_size in output_sizes:
               
        # Update configuration
        config['input_size'] = input_size
        config['output_size'] = 1

        # Load model
        state_dict = torch.load(f'models/mlp/output_size{1}input_size{input_size}.pt')
        mod = tm.train_model(config, it_amt=0, model_fnc=ml_models.MLP, data_dir='data_synthetic')
        mod.load_state_dict(state_dict)
        mod.eval()

        # Load data
        dataloader, idx = dp.get_loaded_data(df_test, win_size=config['input_size'], 
                                             outp_size=output_size, batch_size=32, shuffle=False)

        
        preds = recursive_pred(mod, dataloader,output_size)

        
        # Convert to tensor
        preds_tensor = torch.from_numpy(preds).detach()#.numpy()
        
        ys = []
        for batch in dataloader:
            x,y,tx,ty,sensor = batch
            ys.append(y)
            
        ys = torch.concat(ys)


        # Save results
        results.append({
            'model' : 'MLP_recursive',
            'input_size': input_size,
            'output_size': output_size,
            'loss': loss(preds_tensor,ys).detach().item(),
            'r2' : r2_score(ys, preds_tensor),
            'predictions': preds_tensor.tolist(),  # Save as list if needed
        })

        # Also store in df_test
        #df_test[f'mlp_outp{output_size}_inp{input_size}'] = preds_tensor

# Create DataFrame
df_results = pd.DataFrame(results)

Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training


In [4]:
"""results = []

for input_size in input_sizes:
    for output_size in output_sizes:
        
        # Update configuration
        config['input_size'] = input_size
        config['output_size'] = 1

        # Load model
        state_dict = torch.load(f'models/transformer/output_size{1}input_size{input_size}.pt')
        mod = tm.train_model(config, it_amt=0, model_fnc=ml_models.Transformer, data_dir='data_synthetic')
        mod.load_state_dict(state_dict)
        
        mod.eval()

        # Load data
        dataloader, idx = dp.get_loaded_data(df_test, win_size=config['input_size'], 
                                             outp_size=output_size, batch_size=32, shuffle=False)

        preds = recursive_pred(mod, dataloader,output_size, mod_type = 'transformer')
        
        
        # Convert to tensor
        preds_tensor = torch.from_numpy(preds).detach()#.numpy()
        
        ys = []
        for batch in dataloader:
            x,y,tx,ty,sensor = batch
            ys.append(y)
            
        ys = torch.concat(ys)
        
        print(preds_tensor.shape)
        print(ys.shape)


        # Save results
        results.append({
            'model' : 'Transformer_recursive',
            'input_size': input_size,
            'output_size': output_size,
            'loss': loss(preds_tensor,ys).detach().item(),
            'r2' : r2_score(ys, preds_tensor),
            'predictions': preds_tensor.tolist(),  # Save as list if needed
        })

        # Also store in df_test
        #df_test[f'mlp_outp{output_size}_inp{input_size}'] = preds_tensor


# Create DataFrame
df_this = pd.DataFrame(results)
df_results = pd.concat([df_this, df_results])"""

"results = []\n\nfor input_size in input_sizes:\n    for output_size in output_sizes:\n        \n        # Update configuration\n        config['input_size'] = input_size\n        config['output_size'] = 1\n\n        # Load model\n        state_dict = torch.load(f'models/transformer/output_size{1}input_size{input_size}.pt')\n        mod = tm.train_model(config, it_amt=0, model_fnc=ml_models.Transformer, data_dir='data_synthetic')\n        mod.load_state_dict(state_dict)\n        \n        mod.eval()\n\n        # Load data\n        dataloader, idx = dp.get_loaded_data(df_test, win_size=config['input_size'], \n                                             outp_size=output_size, batch_size=32, shuffle=False)\n\n        preds = recursive_pred(mod, dataloader,output_size, mod_type = 'transformer')\n        \n        \n        # Convert to tensor\n        preds_tensor = torch.from_numpy(preds).detach()#.numpy()\n        \n        ys = []\n        for batch in dataloader:\n            x,y,tx,t

In [5]:
results = []

for input_size in input_sizes:
    for output_size in output_sizes:
        
        # Update configuration
        config['input_size'] = input_size
        config['output_size'] = 1

        # Load model
        state_dict = torch.load(f'models/gcn/output_size{1}input_size{input_size}.pt')
        mod = tm.train_gcn(config, it_amt=0, data_dir='data_synthetic')
        mod.load_state_dict(state_dict)
        
        mod.eval()

        # Load data
        dataloader, idx = dp.get_loaded_data(df_test, win_size=config['input_size'], 
                                             outp_size=output_size, batch_size=32, gcn = True, shuffle=False)

        preds = recursive_pred(mod, dataloader,output_size, mod_type = 'gcn')
        
        # Convert to tensor
        preds_tensor = torch.from_numpy(preds).detach()#.numpy()
        
        ys = []
        for batch in dataloader:
            y = np.vstack(batch.y)
            
            ys.append(y)
            
        ys = torch.from_numpy(np.concatenate(ys))

        
        # Save results
        results.append({
            'model' : 'GCN_recursive',
            'input_size': input_size,
            'output_size': output_size,
            'loss': loss(preds_tensor,ys).detach().item(),
            'r2' : r2_score(ys, preds_tensor),
            'predictions': preds_tensor.tolist(),  # Save as list if needed
        })

        # Also store in df_test
        #df_test[f'mlp_outp{output_size}_inp{input_size}'] = preds_tensor


# Create DataFrame
df_this = pd.DataFrame(results)
df_results = pd.concat([df_this, df_results])

Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training


In [6]:
results = []

for input_size in input_sizes:
    for output_size in output_sizes:
        
        # Update configuration
        config['input_size'] = input_size
        config['output_size'] = 1

        # Load model
        state_dict = torch.load(f'models/gru/output_size{1}input_size{input_size}.pt')
        mod = tm.train_model(config, it_amt=0, model_fnc=ml_models.GRU, data_dir='data_synthetic')
        mod.load_state_dict(state_dict)
        
        mod.eval()

        # Load data
        dataloader, idx = dp.get_loaded_data(df_test, win_size=config['input_size'], 
                                             outp_size=output_size, batch_size=32, shuffle=False)

        preds = recursive_pred(mod, dataloader,output_size)
        
        # Convert to tensor
        preds_tensor = torch.from_numpy(preds).detach()#.numpy()
        
        ys = []
        for batch in dataloader:
            x,y,tx,ty,sensor = batch
            ys.append(y)
            
        ys = torch.concat(ys)

        
        # Save results
        results.append({
            'model' : 'GRU_recursive',
            'input_size': input_size,
            'output_size': output_size,
            'loss': loss(preds_tensor,ys).detach().item(),
            'r2' : r2_score(ys, preds_tensor),
            'predictions': preds_tensor.tolist(),  # Save as list if needed
        })

        # Also store in df_test
        #df_test[f'mlp_outp{output_size}_inp{input_size}'] = preds_tensor

# Create DataFrame
df_this = pd.DataFrame(results)
df_results = pd.concat([df_this, df_results])

Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training


In [7]:
results = []

for input_size in input_sizes:
    for output_size in output_sizes:
        
        # Update configuration
        config['input_size'] = input_size
        config['output_size'] = 1

        # Load model
        state_dict = torch.load(f'models/lstm/output_size{1}input_size{input_size}.pt')
        mod = tm.train_model(config, it_amt=0, model_fnc=ml_models.LSTM, data_dir='data_synthetic')
        mod.load_state_dict(state_dict)
        
        mod.eval()

        # Load data
        dataloader, idx = dp.get_loaded_data(df_test, win_size=config['input_size'], 
                                             outp_size=output_size, batch_size=32, shuffle=False)

        
        preds = recursive_pred(mod, dataloader,output_size)
        
        # Convert to tensor
        preds_tensor = torch.from_numpy(preds).detach()#.numpy()
        
        ys = []
        for batch in dataloader:
            x,y,tx,ty,sensor = batch
            ys.append(y)
            
        ys = torch.concat(ys)

        # Save results
        results.append({
            'model' : 'LSTM_recursive',
            'input_size': input_size,
            'output_size': output_size,
            'loss': loss(preds_tensor, ys).detach().item(),
            'r2' : r2_score(ys, preds_tensor),
            'predictions': preds_tensor.tolist(),  # Save as list if needed
        })

        # Also store in df_test
        #df_test[f'mlp_outp{output_size}_inp{input_size}'] = preds_tensor

# Create DataFrame
df_this = pd.DataFrame(results)
df_results = pd.concat([df_this, df_results])

Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training


In [8]:
results = []

for input_size in input_sizes:
    for output_size in output_sizes:
        
        # Update configuration
        config['input_size'] = input_size
        config['output_size'] = 1

        # Load model
        state_dict = torch.load(f'models/cnn/output_size{1}input_size{input_size}.pt')
        mod = tm.train_model(config, it_amt=0, model_fnc=ml_models.CNN, data_dir='data_synthetic')
        mod.load_state_dict(state_dict)

        # Load data
        dataloader, idx = dp.get_loaded_data(df_test, win_size=config['input_size'], 
                                             outp_size=output_size, batch_size=32, shuffle=False)

        preds = recursive_pred(mod, dataloader,output_size)
        
        # Convert to tensor
        preds_tensor = torch.from_numpy(preds).detach()#.numpy()
        
        ys = []
        for batch in dataloader:
            x,y,tx,ty,sensor = batch
            ys.append(y)
            
        ys = torch.concat(ys)

        # Save results
        results.append({
            'model' : 'CNN_recursive',
            'input_size': input_size,
            'output_size': output_size,
            'loss': loss(preds_tensor,ys).detach().item(),
            'r2' : r2_score(ys, preds_tensor),
            'predictions': preds_tensor.tolist(),  # Save as list if needed
        })

        # Also store in df_test
        #df_test[f'mlp_outp{output_size}_inp{input_size}'] = preds_tensor

# Create DataFrame
df_this = pd.DataFrame(results)
df_results = pd.concat([df_this, df_results])

Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training


In [9]:
results = []

for input_size in input_sizes:
    for output_size in output_sizes:
        
        # Update configuration
        config['input_size'] = input_size
        config['output_size'] = 1

        # Load model
        state_dict = torch.load(f'models/baseline/output_size{1}input_size{input_size}.pt')
        mod = tm.train_model(config, it_amt=0, model_fnc=ml_models.Baseline, data_dir='data_synthetic')
        mod.load_state_dict(state_dict)

        # Load data
        dataloader, idx = dp.get_loaded_data(df_test, win_size=config['input_size'], 
                                             outp_size=output_size, batch_size=32, shuffle=False)

        preds = recursive_pred(mod, dataloader,output_size)
        
        # Convert to tensor
        preds_tensor = torch.from_numpy(preds).detach()#.numpy()
        
        ys = []
        for batch in dataloader:
            x,y,tx,ty,sensor = batch
            ys.append(y)
            
        ys = torch.concat(ys)


        # Save results
        results.append({
            'model' : 'Baseline_recursive',
            'input_size': input_size,
            'output_size': output_size,
            'loss': loss(preds_tensor, ys).detach().item(),
            'r2' : r2_score(ys, preds_tensor),
            'predictions': preds_tensor.tolist(),  # Save as list if needed
        })

        # Also store in df_test
        #df_test[f'mlp_outp{output_size}_inp{input_size}'] = preds_tensor

# Create DataFrame
df_this = pd.DataFrame(results)
df_results = pd.concat([df_this, df_results])

Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training


In [12]:
df_results

,model,input_size,output_size,loss,r2,predictions
0,Baseline_recursive,5,5,0.267927,0.715866,"[[1.099663496017456, 1.0919311046600342, 1.007..."
1,Baseline_recursive,5,10,0.274108,0.696406,"[[1.099663496017456, 1.0919311046600342, 1.007..."
2,Baseline_recursive,5,20,0.293724,0.650273,"[[1.099663496017456, 1.0919311046600342, 1.007..."
3,Baseline_recursive,5,40,0.336407,0.554643,"[[1.099663496017456, 1.0919311046600342, 1.007..."
4,Baseline_recursive,10,5,0.244151,0.740569,"[[1.2792531251907349, 1.3027280569076538, 1.28..."
...,...,...,...,...,...,...
11,MLP_recursive,20,40,0.312081,0.584994,"[[1.1928584575653076, 1.2058427333831787, 1.16..."
12,MLP_recursive,40,5,0.238310,0.744593,"[[1.2438881397247314, 1.3076837062835693, 1.24..."
13,MLP_recursive,40,10,0.245662,0.724117,"[[1.2438881397247314, 1.3076837062835693, 1.24..."
14,MLP_recursive,40,20,0.269570,0.673055,"[[1.2438881397247314, 1.3076837062835693, 1.24..."


In [13]:
df_results.to_csv('results/model_recursive.csv')